In [1]:
import pandas as pd

from maml.models import AtomSets
from maml.describers import SiteElementProperty


# the data contains the MP formation energy data 
# 100 formula - formation energy 
df = pd.read_csv("./mp_ef_100.csv")


## Use MEGNet element features for MP Ef regression

In [2]:
from pymatgen.core import Element

from megnet.data.crystal import get_elemental_embeddings
embeddings = get_elemental_embeddings()
embeddings = {Element(i).Z: j for i, j in embeddings.items()}

In [3]:
describer = SiteElementProperty(embeddings, output_weights=True)
model = AtomSets(describer=describer,  # output weights gives element fractions
                 is_embedding=False,  # we are using the pre-trained element embeddings
                 compile_metrics=["mae"],
                 loss="mse",
                 is_classification=False
                )


2021-10-28 09:11:21.677552: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
features = describer.transform(df['formula'][:80])

In [5]:
model.fit(features, df['Ef'][:80], epochs=10)

2021-10-28 09:11:21.883447: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
1/1 [==============================] - 1s 541ms/step - loss: 3.4798 - mae: 1.8467
Epoch 2/10
1/1 [==============================] - 0s 8ms/step - loss: 3.2928 - mae: 1.7956
Epoch 3/10
1/1 [==============================] - 0s 9ms/step - loss: 3.1086 - mae: 1.7437
Epoch 4/10
1/1 [==============================] - 0s 9ms/step - loss: 2.9282 - mae: 1.6911
Epoch 5/10
1/1 [==============================] - 0s 8ms/step - loss: 2.7443 - mae: 1.6357
Epoch 6/10
1/1 [==============================] - 0s 9ms/step - loss: 2.5555 - mae: 1.5767
Epoch 7/10
1/1 [==============================] - 0s 8ms/step - loss: 2.3596 - mae: 1.5129
Epoch 8/10
1/1 [==============================] - 0s 9ms/step - loss: 2.1565 - mae: 1.4436
Epoch 9/10
1/1 [==============================] - 0s 9ms/step - loss: 1.9437 - mae: 1.3668
Epoch 10/10
1/1 [==============================] - 0s 8ms/step - loss: 1.7230 - mae: 1.2820


In [6]:
test_features = describer.transform(df['formula'][80:])
loss, metric = model.evaluate(test_features, df['Ef'][80:], True)

2/2 [==============================] - 0s 2ms/step - loss: 2.0333 - mae: 1.4135


In [7]:
print(f"The MAE is {metric:.3f} eV/atom")

The MAE is 1.413 eV/atom


## No transfer learning for Ef regression

In [8]:
describer = SiteElementProperty(output_weights=True)
model = AtomSets(describer=describer,
                 is_embedding=True,
                 compile_metrics=["mae"],
                 loss="mse",
                 is_classification=False
                )


In [9]:
features = describer.transform(df['formula'][:80])
model.fit(features, df['Ef'][:80], epochs=10)

Epoch 1/10
1/1 [==============================] - 0s 493ms/step - loss: 3.9591 - mae: 1.9700
Epoch 2/10
1/1 [==============================] - 0s 9ms/step - loss: 3.9043 - mae: 1.9561
Epoch 3/10
1/1 [==============================] - 0s 9ms/step - loss: 3.8614 - mae: 1.9451
Epoch 4/10
1/1 [==============================] - 0s 8ms/step - loss: 3.8193 - mae: 1.9343
Epoch 5/10
1/1 [==============================] - 0s 9ms/step - loss: 3.7755 - mae: 1.9229
Epoch 6/10
1/1 [==============================] - 0s 8ms/step - loss: 3.7284 - mae: 1.9107
Epoch 7/10
1/1 [==============================] - 0s 9ms/step - loss: 3.6765 - mae: 1.8970
Epoch 8/10
1/1 [==============================] - 0s 9ms/step - loss: 3.6190 - mae: 1.8818
Epoch 9/10
1/1 [==============================] - 0s 8ms/step - loss: 3.5558 - mae: 1.8650
Epoch 10/10
1/1 [==============================] - 0s 9ms/step - loss: 3.4863 - mae: 1.8463


In [10]:
test_features = describer.transform(df['formula'][80:])
loss, metric = model.evaluate(test_features, df['Ef'][80:], True)

2/2 [==============================] - 0s 2ms/step - loss: 3.4916 - mae: 1.8587


In [11]:
print(f"The MAE is {metric:.3f} eV/atom")

The MAE is 1.859 eV/atom


## Use MEGNet element features for metallicity classification

In [16]:
df_metal = pd.read_csv("mp_is_metal_100.csv")

In [17]:
df_metal

,formula,is_metal
0,KMnO2,False
1,Cr3Ni,True
2,CsRbAs,True
3,SiSn,False
4,Ca3P2,False
...,...,...
95,K2RhAuF6,False
96,CsK2SbF6,False
97,K2LiTaCl6,True
98,K2NaTaCl6,True


In [18]:
describer = SiteElementProperty(embeddings, output_weights=True)
model = AtomSets(describer=describer,  # output weights gives element fractions
                 is_embedding=False,  # we are using the pre-trained element embeddings
                 compile_metrics=["AUC"],
                 loss="binary_crossentropy",
                 is_classification=True,
                )


In [20]:
features = describer.transform(df['formula'][:80])
model.fit(features, df['is_metal'][:80], epochs=10)

Epoch 1/10
1/1 [==============================] - 1s 733ms/step - loss: 0.7019 - auc: 0.4475
Epoch 2/10
1/1 [==============================] - 0s 8ms/step - loss: 0.6959 - auc: 0.7201
Epoch 3/10
1/1 [==============================] - 0s 8ms/step - loss: 0.6902 - auc: 0.7995
Epoch 4/10
1/1 [==============================] - 0s 9ms/step - loss: 0.6850 - auc: 0.8492
Epoch 5/10
1/1 [==============================] - 0s 8ms/step - loss: 0.6802 - auc: 0.8712
Epoch 6/10
1/1 [==============================] - 0s 9ms/step - loss: 0.6758 - auc: 0.8513
Epoch 7/10
1/1 [==============================] - 0s 8ms/step - loss: 0.6715 - auc: 0.8568
Epoch 8/10
1/1 [==============================] - 0s 8ms/step - loss: 0.6670 - auc: 0.8592
Epoch 9/10
1/1 [==============================] - 0s 9ms/step - loss: 0.6623 - auc: 0.8733
Epoch 10/10
1/1 [==============================] - 0s 8ms/step - loss: 0.6573 - auc: 0.8688


In [21]:
test_features = describer.transform(df['formula'][80:])
loss, metric = model.evaluate(test_features, df['is_metal'][80:], True)

2/2 [==============================] - 0s 3ms/step - loss: 0.6810 - auc: 0.6768


In [22]:
print(f"The AUC is {metric:.3f}")

The AUC is 0.677


## No transfer learning for metallicity classification

In [23]:
describer = SiteElementProperty(output_weights=True)
model = AtomSets(describer=describer,
                 is_embedding=True,
                 compile_metrics=["AUC"],
                 loss="binary_crossentropy",
                 is_classification=True
                )


In [24]:
features = describer.transform(df['formula'][:80])
model.fit(features, df['is_metal'][:80], epochs=10)

Epoch 1/10
1/1 [==============================] - 1s 795ms/step - loss: 0.6944 - auc: 0.5151
Epoch 2/10
1/1 [==============================] - 0s 8ms/step - loss: 0.6915 - auc: 0.5096
Epoch 3/10
1/1 [==============================] - 0s 9ms/step - loss: 0.6896 - auc: 0.8379
Epoch 4/10
1/1 [==============================] - 0s 9ms/step - loss: 0.6877 - auc: 0.7514
Epoch 5/10
1/1 [==============================] - 0s 9ms/step - loss: 0.6856 - auc: 0.8149
Epoch 6/10
1/1 [==============================] - 0s 9ms/step - loss: 0.6832 - auc: 0.8472
Epoch 7/10
1/1 [==============================] - 0s 9ms/step - loss: 0.6807 - auc: 0.8451
Epoch 8/10
1/1 [==============================] - 0s 9ms/step - loss: 0.6780 - auc: 0.8479
Epoch 9/10
1/1 [==============================] - 0s 9ms/step - loss: 0.6749 - auc: 0.8592
Epoch 10/10
1/1 [==============================] - 0s 10ms/step - loss: 0.6716 - auc: 0.8870


In [25]:
test_features = describer.transform(df['formula'][80:])
loss, metric = model.evaluate(test_features, df['is_metal'][80:], True)

2/2 [==============================] - 0s 2ms/step - loss: 0.6924 - auc: 0.5253


In [26]:
print(f"The AUC is {metric:.3f}")

The AUC is 0.525
